### exception test

In [29]:
import os
import ipyparallel as ipp

ipyclient = ipp.Client()
print ipyclient.ids
lbview = ipyclient.load_balanced_view()

[0, 1, 2, 3]


In [93]:
## make data 
data = range(20)

## make function that will raise exception for one data point
@lbview.parallel(block=False)
def testfunc(x):
    import os
    if x[0] < 10: 
        raise Exception("fail")
    else:
        return (x, os.getpid())
    

In [95]:
res = testfunc([3, 30, 40, 3, 4])


In [96]:
[i.get() for i in res]

CompositeError: one or more exceptions from call to method: testfunc
[1:apply]: Exception: fail
[0:apply]: Exception: fail
[3:apply]: Exception: fail

In [70]:
## now run all the data across 4 processors
results = []
try:
    res = lbview.map_async(testfunc, data)
    results = res.get()
    #    results.append(res)
    #ipyclient.wait(results)
    
except Exception:
    print 'hi'
    
finally:
    print 'done'

hi
done


In [71]:
final = [res.get() for i in results]
print len(final), final

0 []
